# Brain Tumor Binary Classifier (2D MRI)

This notebook presents a deep learning pipeline for classifying 2D brain MRI scans into two categories: **Tumor** and **No Tumor**.

The model is built using **PyTorch** and employs a simple convolutional neural network (CNN) architecture tailored for binary classification. Augmentations and preprocessing are applied using **Albumentations**, and the data pipeline leverages **Hugging Face Datasets** for streamlined loading.

The dataset used was originally published on [Kaggle](#) and is re-hosted on the **Hugging Face Hub** with minor modifications to support binary classification.

This project is part of a personal initiative to explore medical imaging with computer vision, focusing on **data preprocessing**, **augmentation techniques**, and **model generalization** in a simplified binary classification task.

> **Disclaimer:** This project is for **educational and research purposes only**. It is *not* intended for medical or clinical use.


## 1. Setup & Dependencies

This section installs and imports the required libraries for data handling, model development, training, and evaluation.


### 1.1 Install Dependencies (Colab Only)

If you're using Google Colab, install the required packages using the command below.


In [ ]:
#install dataset from Huggingface
!pip install -U datasets fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


### 1.2 Import Libraries

We import all necessary libraries including PyTorch, Albumentations, scikit-learn, and Hugging Face datasets.


In [ ]:
# PyTorch core modules
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset

# Torchvision for model architectures and data utilities
from torchvision import datasets
import torchvision.models as models

# Albumentations for data augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Standard libraries
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm
import copy

# Hugging Face datasets
from datasets import load_dataset

## 2. Load and Prepare Raw Dataset

We load a binary brain MRI dataset from the Hugging Face Hub using the `load_dataset` function. This dataset contains two categories and was originally sourced from Kaggle. It is automatically cached for reuse.

The loaded data will be split into training and validation sets using a **stratified sampling** strategy to ensure balanced class representation.


### 2.1 Load Dataset from Hugging Face

We load the dataset directly using the `datasets` library. The dataset contains labeled 2D brain MRI scans across two classes: **tumor** and **no tumor**.


In [ ]:
# Load brain tumor dataset from Hugging Face (auto-cached locally)
ds = load_dataset("Cayanaaa/BrainTumorDatasets", name="binary")
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

binary/train-00000-of-00001.parquet:   0%|          | 0.00/130M [00:00<?, ?B/s]

binary/test-00000-of-00001.parquet:   0%|          | 0.00/25.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5712 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1311 [00:00<?, ? examples/s]

### 2.2 View Class Label Mapping

This command reveals the label names and their corresponding integer encodings used internally by the dataset.


In [ ]:
# Display class labels and their corresponding integer indices
print(ds['train'].features['label'].names)

### 2.3 Extract Images and Labels from Dataset

We extract the raw image and label pairs from the dataset for further processing.


In [ ]:
# Split Data
train_data = ds['train']
images = train_data['image']
labels = train_data['label']

### 2.4 Stratified Train-Validation Split

To ensure balanced class distribution across the training and validation sets, we perform a stratified split. This minimizes the risk of class imbalance during model training.


In [ ]:
train_imgs, val_imgs, train_labels, val_labels = train_test_split(images, labels,
                                                                  test_size=0.2,
                                                                  stratify=labels,
                                                                  random_state=42
                                                                  )

## 3. Dataset Preparation

In this section, we prepare the image dataset by applying preprocessing and augmentation techniques, defining a custom PyTorch `Dataset` class, and creating `DataLoaders` for both training and validation phases.


### 3.1 Define Transformation Pipelines

We define image preprocessing and augmentation pipelines using **Albumentations** to improve generalization and performance.

- The **training pipeline** includes resizing, flipping, distortion, noise, and normalization.
- The **validation pipeline** includes only resizing and normalization to ensure consistent evaluation.


In [ ]:
# Define preprocessing & augmentation for training set
train_T = A.Compose([
    A.Resize(224, 224), # Resize to model input size
    A.HorizontalFlip(p=0.5), # Random horizontal flip
    A.VerticalFlip(p=0.5),  # Random vertical flip
    A.RandomBrightnessContrast(p=0.2),  # Slight brightness/contrast variation
    A.GridDistortion(num_steps=5, distort_limit=0.03, p=0.2),  # Grid-based distortion
    A.GaussNoise(p=0.1), # Add Gaussian noise
    A.Normalize(mean=[0.485, 0.456, 0.406], # Normalize using ImageNet stats
                std=[0.229, 0.224, 0.225]),
    ToTensorV2() # Convert to PyTorch tensor
])

# Define preprocessing for validation set (no augmentation)
val_T = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

### 3.2 Define Custom Dataset Class

We define a custom PyTorch `Dataset` class to:

- Apply the appropriate transformations.
- Return each image and its label in tensor format.


In [ ]:
# Custom Dataset class to load image-label pairs and apply transforms
class LoadDataset(Dataset):
  def __init__(self, image_data, labels, transform=None):
    self.image_data = image_data
    self.labels = labels
    self.transform = transform

  def __len__(self):
    return len(self.image_data)

  def __getitem__(self, idx):
    img = self.image_data[idx]
    img = img.convert('RGB') # Ensure image is in RGB format
    img = np.array(img)

    label = self.labels[idx]

    if self.transform:
      img = self.transform(image=img)['image']

    return img, torch.tensor(label, dtype=torch.long)

### 3.3 Create Dataset & DataLoader

We wrap the image-label pairs using our custom `Dataset` class, and prepare `DataLoaders` to efficiently feed data during training and evaluation.


In [ ]:
# Wrap image and label arrays into Dataset objects
train_dataset = LoadDataset(train_imgs, train_labels, train_T)
val_dataset = LoadDataset(val_imgs, val_labels, val_T)

# Create DataLoaders for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # shuffle for training
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False) # no shuffle for validation

## 4. Model, Optimizer, and Training Setup

We adopt a **transfer learning** approach using a pre-trained **DenseNet121** model. To preserve the visual features learned from ImageNet, all convolutional layers are **frozen**, and we train **only the classifier head**. This initial setup focuses on **feature extraction**, before performing full fine-tuning in a later stage.
Define Models

### 4.1 Load Pre-trained Model

We load **DenseNet121** with ImageNet weights to leverage powerful low-level feature extraction learned from large-scale natural images.


In [ ]:
# Load DenseNet121 model pre-trained on ImageNet
model = models.densenet121(pretrained=True)

# Freeze all layers in the feature extractor to retain pre-trained representations
for param in model.parameters():
  param.requires_grad = False

# Replace the classifier head to match the number of output classes (4)
model.classifier = nn.Linear(model.classifier.in_features, 2)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 124MB/s]


### 4.2 Define Optimizer, Scheduler, and Device

We use the Adam optimizer to update only the classifier head. A learning rate scheduler reduces the learning rate when validation performance plateaus. GPU is used if available.


In [ ]:
# Configure optimizer to update only the classifier head
early_optimizer = torch.optim.Adam(model.classifier.parameters(), lr=1e-3)

# Set up learning rate scheduler to reduce LR if validation loss stops improving
scheduler_early =ReduceLROnPlateau(early_optimizer, mode='min', patience=2, factor=0.1, verbose=True)

# Automatically use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to the selected device
model.to(device)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


### 4.3 Define Weighted Loss Function

To address class imbalance in the training data, we compute class weights and apply them to the cross-entropy loss function.


In [ ]:
# Compute class weights to handle imbalance and reduce bias toward frequent classes
class_weight = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weight = torch.tensor(class_weight, dtype=torch.float32).to(device)

# Define weighted cross-entropy loss
crieterion = nn.CrossEntropyLoss(weight=class_weight)

### 4.4 Define Early Stopping

We implement a custom early stopping mechanism to terminate training when the validation loss no longer improves after a specified number of epochs.


In [ ]:
# Custom early stopping class to monitor validation performance
# Stops training if no improvement is observed over 'patience' epochs
class EarlyStopping:
    def __init__(self, monitor='val_loss', mode='min', patience=3, delta=0.0, verbose=True):
         """
        Args:
            monitor (str): Metric to monitor ('val_loss' or 'val_acc')
            mode (str): 'min' → lower is better, 'max' → higher is better
            patience (int): # of epochs with no improvement before stopping
            delta (float): Minimum change to qualify as improvement
            verbose (bool): Print status each epoch if True
        """
        self.monitor = monitor
        self.mode = mode
        self.patience = patience
        self.delta = delta
        self.verbose = verbose

        self.best_score = None
        self.counter = 0
        self.early_stop = False

        # Set comparison function and initial best value
        if self.mode == 'min':
            self.monitor_op = lambda current, best: current < best - self.delta
            self.best_score = np.inf
        elif self.mode == 'max':
            self.monitor_op = lambda current, best: current > best + self.delta
            self.best_score = -np.inf
        else:
            raise ValueError("mode must be 'min' or 'max'")

    def __call__(self, current_score):
        # Initialize best score
        if self.best_score is None:
            self.best_score = current_score
            if self.verbose:
                print(f"[EarlyStopping] Initial best {self.monitor}: {self.best_score:.4f}")
        # Check for improvement
        elif self.monitor_op(current_score, self.best_score):
            self.best_score = current_score
            self.counter = 0
            if self.verbose:
                print(f"[EarlyStopping] Improved {self.monitor}: {self.best_score:.4f}")
        else:
            self.counter += 1
            if self.verbose:
                print(f"[EarlyStopping] No improvement in {self.monitor} for {self.counter}/{self.patience} epochs.")
            # Stop training if performance has not improved for 'patience' epochs
            if self.counter >= self.patience:
                if self.verbose:
                    print(f"[EarlyStopping] Stopping training. Best {self.monitor}: {self.best_score:.4f}")
                self.early_stop = True

In [ ]:
# Create an EarlyStopping instance to monitor validation loss
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3)

## 5. Train Classifier Head (Warm-up Phase)

In this phase, we only train the classifier head (fully connected layers) while keeping the backbone frozen. This **warm-up strategy** helps the model gradually adapt to the domain-specific brain MRI data without modifying the general features learned from ImageNet.

The goal is to allow the final layers to specialize on our dataset before unfreezing and fine-tuning the entire network.


In [ ]:
# Save initial model weights and set best validation loss to infinity
best_model_wts = copy.deepcopy(model.state_dict())
best_val_loss = np.inf

num_epochs = 100

for epoch in range(num_epochs):
  print(f"-" * 50)
  print(f"\nEpoch {epoch+1}/{num_epochs}")
  print(f"-" * 50)

  # --- Training Phase ---
  model.train()
  train_loss = 0.0
  correct = 0
  total = 0

  for images, labels in tqdm(train_loader, desc="Training"):
    images, labels = images.to(device), labels.to(device)

    early_optimizer.zero_grad()
    outputs = model(images)
    loss = crieterion(outputs, labels)
    loss.backward()
    early_optimizer.step()

    train_loss += loss.item() * images.size(0)
    _, predicted = torch.max(outputs, dim=1)
    correct += (predicted == labels).sum().item()
    total += labels.size(0)

  avg_train_loss = train_loss / total
  train_acc = correct / total

  # --- Validation Phase ---
  model.eval()
  val_loss = 0.0
  correct = 0
  total = 0

  with torch.no_grad():
    for images, labels in tqdm(val_loader, desc="Validation"):
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      loss = crieterion(outputs, labels)

      val_loss += loss.item() * images.size(0)
      _, predicted = torch.max(outputs, dim=1)
      correct += (predicted == labels).sum().item()
      total += labels.size(0)

    avg_val_loss = val_loss / total
    val_acc = correct / total

    # ==== output per epoch ====
    print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Step the learning rate scheduler and update early stopping
    scheduler_early.step(avg_val_loss)
    early_stopping(avg_val_loss)

    # Save model weights if validation loss improves
    if avg_val_loss < best_val_loss:
      best_val_loss = avg_val_loss
      best_model_wts = copy.deepcopy(model.state_dict())
      torch.save(model.state_dict(), 'best_model.pth')
      print(f"[INFO]: Best Model Updated")

    # Stop training if early stopping is triggered
    if early_stopping.early_stop:
      print(f"[INFO]: Training stopped by early stopping")
      break

# Load best model weights after training
model.load_state_dict(best_model_wts)
print(f"[INFO]: Best model loaded")



Epoch 1/100
----------------------------------------


Validation: 100%|██████████| 18/18 [00:05<00:00,  3.02it/s]


Train Loss: 0.3485 | Train Acc: 0.8693
Val Loss: 0.1780 | Val Acc: 0.9501
[EarlyStopping] Improved val_loss: 0.1780
[INFO]: Best Model Updated

Epoch 2/100
----------------------------------------


Validation: 100%|██████████| 18/18 [00:05<00:00,  3.19it/s]


Train Loss: 0.1950 | Train Acc: 0.9392
Val Loss: 0.1303 | Val Acc: 0.9641
[EarlyStopping] Improved val_loss: 0.1303
[INFO]: Best Model Updated

Epoch 3/100
----------------------------------------


...

Train Loss: 0.0922 | Train Acc: 0.9700
Val Loss: 0.0858 | Val Acc: 0.9781
[EarlyStopping] No improvement in val_loss for 1/3 epochs.

Epoch 18/100
----------------------------------------


Validation: 100%|██████████| 18/18 [00:06<00:00,  2.94it/s]


Train Loss: 0.0918 | Train Acc: 0.9691
Val Loss: 0.0861 | Val Acc: 0.9781
[EarlyStopping] No improvement in val_loss for 2/3 epochs.

Epoch 19/100
----------------------------------------


Validation: 100%|██████████| 18/18 [00:05<00:00,  3.02it/s]

Train Loss: 0.0984 | Train Acc: 0.9663
Val Loss: 0.1042 | Val Acc: 0.9720
[EarlyStopping] No improvement in val_loss for 3/3 epochs.
[EarlyStopping] Stopping training. Best val_loss: 0.0835
[INFO]: Training stopped by early stopping
[INFO]: Best model loaded


## 6. Fine-Tuning Setup

In this phase, we fine-tune the deeper parts of the model to better adapt to the brain tumor classification task. Instead of unfreezing the entire backbone, we selectively unfreeze the final convolutional block and normalization layer to balance adaptability and generalization.

Fine-tuning allows the model to refine high-level features learned from ImageNet in a domain-specific context.

Fine Tuning

### 6.1 Unfreeze Selected Layers

Here, we unfreeze the `denseblock4` and `norm5` layers of the backbone while keeping all earlier layers frozen. This selective unfreezing helps avoid overfitting and reduces the risk of catastrophic forgetting.



In [ ]:
# Only unfreeze the last DenseBlock and final batch norm layer (norm5)
for name, layer in model.named_parameters():
  if 'denseblock4' in name or 'norm5' in name:
    param.requires_grad = True
  else:
    param.requires_grad = False

### 6.2 Fine-Tuning Optimizer & Callbacks

We define a new optimizer and learning rate scheduler for the fine-tuning phase. Only the parameters marked as trainable (i.e., from `denseblock4` and `norm5`) are updated during this phase.

An `EarlyStopping` callback is also set up to prevent overfitting by halting training when the validation loss no longer improves.

> **Note**: We print the active learning rate after optimizer setup to verify that the new learning rate is properly configured.


In [ ]:
# Define optimizer for fine-tuning (only trainable parameters)
ft_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

# Print current learning rate (for verification)
current_lr = ft_optimizer.param_groups[0]['lr']
print(f"Active learning rate: {current_lr}")

# Define scheduler for fine-tuning
scheduler_ft = ReduceLROnPlateau(ft_optimizer, mode='min', patience=2, factor=0.1, verbose=True)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=3, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


## 7. Fine-Tune Backbone (Training Loop)

In this section, we perform **fine-tuning** by training the previously unfrozen layers (`denseblock4` and `norm5`) along with the classifier head. Unlike the warm-up phase, this step allows the model to adjust higher-level convolutional features to the specific patterns present in brain MRI images.

The training loop here follows the same structure as the warm-up phase (Section 5), with updated optimizer and scheduler settings defined in Section 6.2. We continue to monitor validation loss and apply **early stopping** to prevent overfitting.


In [ ]:
# Save initial model weights and set best validation loss to infinity
best_model_wts = copy.deepcopy(model.state_dict())
best_val_loss = np.inf

for epoch in range(num_epochs):
  print(f"-" * 50)
  print(f"Epoch {epoch+1}/{num_epochs}")
  print(f"-" * 50)

  # --- Train Phases ---
  model.train()
  train_loss, correct, total = 0.0, 0, 0

  for images, labels in tqdm(train_loader, desc="Training"):
    images, labels = images.to(device), labels.to(device) # add images, labels ke device (gpu)

    ft_optimizer.zero_grad() # reset gradient before backward pass to prevent accumulation
    outputs = model(images)
    loss = crieterion(outputs, labels)
    loss.backward() # calculate the gradient of the loss
    ft_optimizer.step() # update weight based on gradient

    train_loss += loss.item() * images.size(0)
    _, predicted = torch.max(outputs, dim=1) # take class prediction (argmax) from model output
    correct += (predicted == labels).sum().item()
    total += labels.size(0)

  avg_train_loss = train_loss / total
  train_acc = correct / total

  # --- Validation Phases ---
  model.eval() # enter eval mode: dropout, batchnorm will be deactive
  val_loss, correct, total = 0.0, 0, 0

  with torch.no_grad(): # Disable gradient calculations to make interfaces faster & more efficient
    for images, labels in tqdm(val_loader, desc="Validation"):
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      loss = crieterion(outputs, labels)

      val_loss += loss.item() * images.size(0)
      _, predicted = torch.max(outputs, dim=1)
      correct += (predicted == labels).sum().item()
      total += labels.size(0)

    avg_val_loss = val_loss / total
    val_acc = correct / total

    print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}")

    scheduler_ft.step(avg_val_loss)
    early_stopping(avg_val_loss)

    if avg_val_loss < best_val_loss:
      best_val_loss = avg_val_loss
      best_model_wts = copy.deepcopy(model.state_dict())
      torch.save(model.state_dict(), "best_model_fineTuning.pth")
      print(f"[INFO]: Best model FineTuning Updated")

    if early_stopping.early_stop:
      print(f"[INFO]: Training Stopped by early stopping")
      break

# Load best model weights after training
model.load_state_dict(best_model_wts)
print(f"[INFO]: Best model from FineTuning Loaded")

Epoch 1/100
----------------------------------------


Validation: 100%|██████████| 18/18 [00:05<00:00,  3.19it/s]


Train Loss: 0.0849 | Train Acc: 0.9698
Val Loss: 0.0831 | Val Acc: 0.9773
[EarlyStopping] Improved val_loss: 0.0831
[INFO]: Best model FineTuning Updated
Epoch 2/100
----------------------------------------


Validation: 100%|██████████| 18/18 [00:05<00:00,  3.11it/s]


Train Loss: 0.0937 | Train Acc: 0.9702
Val Loss: 0.0833 | Val Acc: 0.9764
[EarlyStopping] No improvement in val_loss for 1/3 epochs.
Epoch 3/100
----------------------------------------


Validation: 100%|██████████| 18/18 [00:06<00:00,  3.00it/s]


Train Loss: 0.0962 | Train Acc: 0.9685
Val Loss: 0.0832 | Val Acc: 0.9764
[EarlyStopping] No improvement in val_loss for 2/3 epochs.
Epoch 4/100
----------------------------------------


Validation: 100%|██████████| 18/18 [00:05<00:00,  3.19it/s]

Train Loss: 0.1032 | Train Acc: 0.9654
Val Loss: 0.0841 | Val Acc: 0.9764
[EarlyStopping] No improvement in val_loss for 3/3 epochs.
[EarlyStopping] Stopping training. Best val_loss: 0.0831
[INFO]: Training Stopped by early stopping
[INFO]: Best model from FineTuning Loaded


## 8. Save Final Model

After fine-tuning, the best-performing model (based on validation loss) is saved using `torch.save()`. This ensures that the most generalizable version of the model is preserved for deployment or further evaluation.

For privacy and reproducibility, the model is uploaded to Hugging Face Hub instead of being stored in a local path. The download link or model reference will be provided in the project source files.

**Model location**: Refer to the model card or config file in the `src/` directory.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.save(model.state_dict(), '/path/to/your/drive')

## 📌 Conclusion

This notebook documents an early-stage exploration into **medical image classification** using deep learning.

By applying **transfer learning** with **DenseNet121** to a binary brain tumor dataset, I gained practical experience in designing data pipelines, implementing image augmentations, and managing training workflows in **PyTorch**.

The process offered valuable insights into **model behavior**, **class distribution handling**, and the importance of **reproducible experiments** — all within a manageable yet meaningful problem space.

This project serves as a foundational step in my journey through **computer vision**, and sets the stage for more complex tasks, such as **multiclass classification**, **model interpretability**, and eventual **deployment** in real-world scenarios.

> 💡 This notebook is part of a progressive deep learning portfolio. Each step builds toward stronger modeling capabilities and a deeper understanding of applied AI in medical contexts.
